## Download Required Dependencies

In [ ]:
import subprocess
import sys
import os
# Function to install a package using pip
def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of required packages
required_packages = [
    "scikit-learn",
    "scikit-image",
    "numpy",
    "matplotlib",
    "opencv-python",
    "roboflow",
    "pandas",
]

# Check and install each package
for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        print(f"Installing {package}...")
        install_package(package)

## Implementation of HOG Feature Descriptor

Implementation of HOG Feature Descriptor involves the following steps:
1. Preprocess the data
2. Calculate and Visualize the Histogram of Gradients


### Step 1: Preprocess the data

In [ ]:
from skimage.io import imread, imshow
# from skimage.transform import resize
import skimage
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
import numpy as np
import math
import cv2

image_BGR = cv2.imread("./media/puppy.jpg")
image_RGB = cv2.cvtColor(image_BGR, cv2.COLOR_BGR2RGB)

gray_image = cv2.cvtColor(image_BGR, cv2.COLOR_BGR2GRAY) 

resized_image = cv2.resize(image_RGB, (64, 128))
resized_gray_image = cv2.resize(gray_image, (64, 128))

fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes[0, 0].imshow(image_RGB)
axes[0, 1].imshow(resized_image)
axes[1, 0].imshow(gray_image, cmap='gray')
axes[1, 1].imshow(resized_gray_image, cmap='gray')


### Step 2: Calculate and Visualize the Histogram of Gradients


In [ ]:
from skimage.feature import hog
from skimage import exposure

import cv2

fd, hog_image = hog(resized_gray_image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(8, 4), sharex=True, sharey=True)

ax1.imshow(resized_image)
ax1.set_title('Input image')

ax2.imshow(resized_gray_image, cmap=plt.cm.gray)
ax2.set_title('Grayscale image') 

ax3.imshow(hog_image, cmap=plt.cm.gray)
ax3.set_title('Histogram of Oriented Gradients') 

## Implementation of eXtended Center-Symmetric Local Binary Pattern (XCS-LBP) Feature Descriptor

### Step 1: Calculate XCS-LBP operator applied to the whole grayscale image

In [4]:
image_BGR = cv2.imread("./media/test_image.png")
image_RGB = cv2.cvtColor(image_BGR, cv2.COLOR_BGR2RGB)
gray_image = cv2.cvtColor(image_RGB, cv2.COLOR_BGR2GRAY) 

def valid_index(image, x, y):
    return (x >= 0 and y >= 0 and x < image.shape[0] and y < image.shape[1])

def threshold_function(x):
    return 1 if x >= 0 else 0

def calculate_xcs_lbp_value(x, y, image, neighborhood_size=8):
    result = 0
    mask = [(-1, -1), (-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1)]
    half_neighborhood_size = int(neighborhood_size/2)
    image=image.astype('float32')
    for i in range(half_neighborhood_size):
        index1 = (x+mask[i][0], y+mask[i][1])
        index2 = (x+mask[i+half_neighborhood_size][0], y+mask[i+half_neighborhood_size][1])
        value1 = image[index1[0], index1[1]] if valid_index(image, index1[0], index1[1]) else 0
        value2 = image[index2[0], index2[1]] if valid_index(image, index2[0], index2[1]) else 0

        g1 = (value1-value2) + image[x, y]
        g2 = (value1-image[x, y])*(value2-image[x, y])
        power_val = [1, 2, 4, 8, 16, 32, 64, 128]
        result += (threshold_function(g1+g2)*power_val[i])
    return result

xcs_lbp_image = gray_image.copy()
for i in range(xcs_lbp_image.shape[0]):
    for j in range(xcs_lbp_image.shape[1]):
        xcs_lbp_image[i, j] = calculate_xcs_lbp_value(i, j, gray_image)


In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12,8))
ax1.imshow(image_RGB)
ax1.set_title("Input Image")
ax2.imshow(gray_image, cmap='gray')
ax2.set_title("Grayscale Image")
ax3.imshow(xcs_lbp_image, cmap='gray')
ax3.set_title("XCS-LBP")


In [6]:
import pandas as pd

dataset_folder = "./INRIA_Person_detection_dataset"
dataset_name_list = ["train", "test", "valid"]
data_folder = "./data"

X = {}
y = {}

for dataset_name in dataset_name_list:
    if not(os.path.exists(data_folder)):
        os.mkdir(data_folder)
    features_path = os.path.join(data_folder, "extracted_features")
    if not(os.path.exists(features_path)):
        os.mkdir(features_path)
    features_path = os.path.join(features_path, dataset_name)
    if not(os.path.exists(features_path)):
        os.mkdir(features_path)
    labeled_data_path = os.path.join(features_path, "HOG_features.npy")
    if not(os.path.exists(labeled_data_path)):
        dataset_path = os.path.join(dataset_folder, dataset_name)
        annotations_path = os.path.join(dataset_path, "_annotations.csv")
        annotations = pd.read_csv(annotations_path)
        pair_list = []
        hog_image_list = []
        for index, row in annotations.iterrows():
            input_image = cv2.imread(os.path.join(dataset_path, row["filename"]))
            grayscale_input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
            cropped_image = grayscale_input_image[row["ymin"]: row["ymax"], row["xmin"]: row["xmax"]]
            cropped_image = cv2.resize(cropped_image, (64, 128))

            features, hog_image = hog(cropped_image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
            pair_list.append((features, 1 if row["class"]=="person" else 0))
            hog_image_list.append((hog_image, 1 if row["class"]=="person" else 0))

        np.save(os.path.join(features_path, "HOG_features"), np.asarray(pair_list, dtype="object"))
        np.save(os.path.join(features_path, "HOG_images"), np.asarray(hog_image_list, dtype="object"))
    
    labeled_data = np.load(labeled_data_path, allow_pickle=True)
    X[dataset_name] = [pair[0] for pair in labeled_data]
    y[dataset_name] = [pair[1] for pair in labeled_data] 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
rf = RandomForestClassifier(n_estimators=100, criterion="gini", max_features="sqrt", random_state=42)
rf.fit(X["train"], y["train"])

test_report = classification_report(y["test"], rf.predict(X["test"]))
print(test_report)

In [ ]:
from sklearn import svm

model = svm.SVC(kernel='linear')
model.fit(X["train"], y["train"])

test_report = classification_report(y["test"], model.predict(X["test"]))
print(test_report)

In [ ]:
dataset_folder = "./INRIA_Person_detection_dataset"
dataset_name_list = ["train", "test", "valid"]
data_folder = "./data"

X = {}
y = {}
A = {}
b = {}

for dataset_name in dataset_name_list:
    if not(os.path.exists(data_folder)):
        os.mkdir(data_folder)
    features_path = os.path.join(data_folder, "extracted_features")
    if not(os.path.exists(features_path)):
        os.mkdir(features_path)
    features_path = os.path.join(features_path, dataset_name)
    if not(os.path.exists(features_path)):
        os.mkdir(features_path)
    labeled_data_path = os.path.join(features_path, "XCS_LBP_features.npy")
    if not(os.path.exists(labeled_data_path)):
        dataset_path = os.path.join(dataset_folder, dataset_name)
        annotations_path = os.path.join(dataset_path, "_annotations.csv")
        annotations = pd.read_csv(annotations_path)
        pair_list = []
        xcs_lbp_image_list = []
        for index, row in annotations.iterrows():
            input_image = cv2.imread(os.path.join(dataset_path, row["filename"]))
            grayscale_input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
            cropped_image = grayscale_input_image[row["ymin"]: row["ymax"], row["xmin"]: row["xmax"]]
            cropped_image = cv2.resize(cropped_image, (128, 256))
            xcs_lbp_image = cropped_image.copy()

            for i in range(xcs_lbp_image.shape[0]):
                for j in range(xcs_lbp_image.shape[1]):
                    xcs_lbp_image[i, j] = calculate_xcs_lbp_value(i, j, cropped_image)

            features = []
            for i in range(xcs_lbp_image.shape[0]):
                features.extend(xcs_lbp_image[i])

            pair_list.append((features, 1 if row["class"]=="person" else 0))
            xcs_lbp_image_list.append((xcs_lbp_image, 1 if row["class"]=="person" else 0))

        np.save(os.path.join(features_path, "XCS_LBP_features"), np.asarray(pair_list, dtype="object"))
        np.save(os.path.join(features_path, "XCS_LBP_images"), np.asarray(xcs_lbp_image_list, dtype="object"))
    
    labeled_data = np.load(labeled_data_path, allow_pickle=True)
    X[dataset_name] = [pair[0] for pair in labeled_data]
    y[dataset_name] = [pair[1] for pair in labeled_data] 

    visualized_data_path = os.path.join(features_path, "XCS_LBP_images.npy")
    visualized_data = np.load(visualized_data_path, allow_pickle=True)
    
    A[dataset_name] = [pair[0] for pair in visualized_data]
    b[dataset_name] = [pair[1] for pair in visualized_data] 


dataset_path = os.path.join(dataset_folder, "valid")
annotations_path = os.path.join(dataset_path, "_annotations.csv")
annotations = pd.read_csv(annotations_path)
for index, row in annotations.iterrows():
    if index == 201:
        input_image = cv2.imread(os.path.join(dataset_path, row["filename"]))
        grayscale_input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
        cropped_image = grayscale_input_image[row["ymin"]: row["ymax"], row["xmin"]: row["xmax"]]

        xcs_lbp_image = cropped_image.copy()

        for i in range(xcs_lbp_image.shape[0]):
            for j in range(xcs_lbp_image.shape[1]):
                xcs_lbp_image[i, j] = calculate_xcs_lbp_value(i, j, cropped_image)
        test_image = cropped_image
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,8))
        ax1.imshow(test_image, cmap='gray')
        ax1.set_title("Input Image")
        ax2.imshow(A["valid"][index], cmap='gray')
        ax2.set_title("XCS-LBP")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
rf = RandomForestClassifier(n_estimators=100, criterion="entropy", max_features="log2", random_state=42)
rf.fit(X["train"], y["train"])

test_report = classification_report(y["test"], rf.predict(X["test"]))
print(test_report)

In [ ]:
from sklearn import svm
from sklearn.metrics import classification_report

model = svm.SVC(kernel='linear')
model.fit(X["train"], y["train"])

test_report = classification_report(y["test"], model.predict(X["test"]))
print(test_report)